In [ ]:
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
# Local modules
sys.path.append('..')
import reproducibility
from utils import print_dict_summary
from data_funcs import load_and_fix_data, rmse
from abc import ABC, abstractmethod
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import mean_squared_error
from utils import hash2

In [ ]:
reproducibility_file='version_control/reproducibility_dict0.pickle'

repro={}
repro.update(load_and_fix_data(reproducibility_file))
print_dict_summary(repro)

case = 'case11'
case_data=repro[case]
case_data["h2"]=300

## Stateful Batch Training

In [ ]:
from moisture_rnn import RNN, create_rnn_data2
import logging
from utils import logging_setup
from moisture_rnn_pkl import pkl2train
logging_setup()

In [ ]:
reproducibility_file='data/reproducibility_dict2.pickle'
train = pkl2train([reproducibility_file])

In [ ]:
import yaml

with open("params.yaml") as file:
    params = yaml.safe_load(file)["rnn_repro"]
params

In [ ]:

rnn_dat = create_rnn_data2(train["reproducibility"], params)

In [ ]:
# Update Params for Reproducibility

params.update({
    'epochs':200,
    'dropout': [0, 0], # NOTE: length must match total number of layers, default is 1 hidden recurrent layer and 1 dense output layer
    'recurrent_dropout': 0, # Length must match number of recurrent layers    
    'rnn_units': 20
})

In [ ]:
reproducibility.set_seed()
rnn = RNN(params)

In [ ]:
rnn.fit(rnn_dat["X_train"], rnn_dat["y_train"])
# rnn.fit(X_train, y_train)

In [ ]:
# preds = rnn.predict(np.reshape(X,(1, X.shape[0], features)))
print(hash2(rnn_dat['X']))
print(hash2(rnn_dat['y']))
preds = rnn.predict(rnn_dat['X'])
rmse(preds, rnn_dat['y'])

In [ ]:
reproducibility.set_seed()
rnn = RNN(params)
rnn.run_model(rnn_dat)

## Physics Initialized

---

## New Developments

### Other Hyperparams

In [ ]:
params.update({
    'activation': ['sigmoid', 'relu'], # Length must match total number of layers
    'dropout': [0.2, 0.2], # NOTE: length must match total number of layers, default is 1 hidden recurrent layer and 1 dense output layer
    'recurrent_dropout': 0.2, # Length must match number of recurrent layers
    'learning_rate': 0.003,
    'rnn_units': 9,
    'epochs': 100
})

In [ ]:
reproducibility.set_seed()
rnn = RNN(params)

In [ ]:
rnn.fit(rnn_dat["X_train"], rnn_dat["y_train"], verbose_fit=False)

In [ ]:
preds = rnn.predict(rnn_dat["X"])
# np.sqrt(mean_squared_error(preds.flatten(), rnn_dat["y"].flatten())
rmse(preds, rnn_dat["y"])

### Validation Error

In [ ]:
params.update({
    'train_frac': 0.5,
    'val_frac': 0.1
})
rnn_dat = create_rnn_data2(train["reproducibility"], params)

In [ ]:
reproducibility.set_seed()
rnn = RNN(params)
rnn.fit(rnn_dat["X_train"], rnn_dat["y_train"], validation_data = (rnn_dat["X_val"], rnn_dat["y_val"]))

In [ ]:
import importlib
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import RNN

In [ ]:
reproducibility.set_seed()
rnn = RNN(params)
rnn.run_model(rnn_dat)